### Render stac outputs with mujoco

In [14]:
%load_ext autoreload
%autoreload 2

# glfw is faster, but use osmesa if glfw not available
%env MUJOCO_GL=glfw
%env PYOPENGL_PLATFORM=glfw

import os
import mediapy as media

from stac_mjx.viz import viz_stac
from stac_mjx import main
from stac_mjx import utils
from pathlib import Path

# Disable jax's memory preallocation if you're running multiple notebooks using jax
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false" 

base_path = Path.cwd().parent
stac_config_path = base_path / "demos/demo_stac.yaml"
model_config_path = base_path / "configs/rodent.yaml"

stac_cfg, model_cfg = main.load_configs(stac_config_path, model_config_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: MUJOCO_GL=glfw
env: PYOPENGL_PLATFORM=glfw


In [15]:
# set args and initialize params
data_path = base_path / "demo_fit.p"
n_frames = 10
save_path=base_path / "videos/direct_render.mp4"

# Call mujoco_viz
frames = viz_stac(data_path, stac_cfg, model_cfg, n_frames, save_path, start_frame=0, camera="close_profile", base_path=Path.cwd().parent)

# Show the video in the notebook (it is also saved to the save_path)
media.show_video(frames, fps=model_cfg["RENDER_FPS"])

0it [00:00, ?it/s]/Users/charleszhang/miniforge3/envs/stac-mjx/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
10it [00:00, 15.44it/s]


This browser does not support the video tag.
